# Датасет

**Heart Attack Analysis & Prediction Dataset**

A dataset for heart attack classification

[Ссылка на kaggle](https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset)